This cell is to mount my Google Cloud Disk, you don’t need to run it during testing

In [2]:
from google.colab import drive
drive.mount('/content/drive')

import os

path = "/content/drive/My Drive/yelp-rs/dataset/"
os.chdir(path)
os.listdir(path)
!ls

Mounted at /content/drive
covid_19_dataset.tar
last_1_years_restaurant_reviews.csv
last_1_years_restaurant_reviews.gsheet
yelp_academic_dataset_business.json
yelp_academic_dataset_covid_features.json
yelp_academic_dataset_review.json


In [4]:
!pip install surprise

     |████████████████████████████████| 11.8MB 578kB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp36-cp36m-linux_x86_64.whl size=1618265 sha256=95c2ed3290f8fbfc69971c548877b5336378819587b5214c8a11c0d96ed94f39
  Stored in directory: /root/.cache/pip/wheels/78/9c/3d/41b419c9d2aff5b6e2b4c0fc8d25c538202834058f9ed110d0
Successfully built scikit-surprise


**Data Processing**

This part is data preprocessing. 
Due to the large amount of data, I have submitted the processed last_1_years_restaurant_reviews.csv file together, and you don’t need to run it during testing.

In [ ]:
import json
import pandas as pd
from tqdm import tqdm


def process_business(df_business):

    # city, check if it contains "Las Vegas"
    filter_city = df_business['city'].apply(str).str.contains("Las Vegas")
    # categories, check if it is null
    filter_category_not_null = ~df_business["categories"].isnull()
    # categories, check if it contains "Restaurants"
    filter_category_restaurant = df_business["categories"].apply(str).str.contains("Restaurants")
    # filter DataFrame, and name it df_filtered
    df_filtered = df_business[filter_city & filter_category_not_null & filter_category_restaurant]
    # keep relevant columns
    df_selected_business = df_filtered[['business_id', 'name', 'categories', 'stars']]
    # Rename
    df_selected_business.rename(columns={"stars":"avg_stars"}, inplace=True)
    # set index
    df_selected_business = df_selected_business.set_index('business_id')

    return df_selected_business


def merge_review(df_review, df_selected_business):

    # filter. You might want to alter the datetime, in order to get more data
    df_review_filtered = df_review[df_review['date']>'2019-10-1'].set_index('business_id')
    # inner join df_left and df_right
    df_final = df_selected_business.join(df_review_filtered, how='inner')
    # reset the index 
    df_final = df_final.reset_index()

    return df_final


if __name__ == "__main__":

    # read business data into DataFrame
    print ("Read business yelp dataset...")
    total_business_lines = sum(1 for line in open("yelp_academic_dataset_business.json",'r', encoding='UTF-8'))
    with open("yelp_academic_dataset_business.json",'r', encoding='UTF-8') as f:
        df_business = pd.DataFrame(json.loads(line) for line in tqdm(f, total=total_business_lines))

    # read review data. 
    print ("Read review yelp dataset...(might take up to 5 mins)")
    total_review_lines = sum(1 for line in open('yelp_academic_dataset_review.json','r', encoding='UTF-8'))
    with open('yelp_academic_dataset_review.json','r', encoding='UTF-8') as f:
        df_review = pd.DataFrame(json.loads(line) for line in tqdm(f, total=total_review_lines))

    # process business
    df_selected_business = process_business(df_business)

    print ("Start merging business and review data...")
    # merge data
    df_final = merge_review(df_review, df_selected_business)

    # write tmp data
    # Save to data/last_1_years_restaurant_reviews.csv for your next task
    df_final.to_csv('last_1_years_restaurant_reviews.csv',index=False)
    print("Sucess")


In [ ]:
df_final.head()

COVID19_DATASET

In [ ]:
import json
import pandas as pd
from tqdm import tqdm
line_count = len(open("yelp_academic_dataset_covid_features.json",'r', encoding='UTF-8').readlines())
business_ids, highlights, delivery_or_takeout, Grubhub_enabled, Call_To_Action_enabled, Request_a_Quote_Enabled, Covid_Banner, Temporary_Closed_Until, Virtual_Services_Offered = [],[],[],[],[],[],[],[],[]
with open("yelp_academic_dataset_covid_features.json",'r', encoding='UTF-8') as f:
    for line in tqdm(f, total=line_count):
        blob = json.loads(line)
        business_ids += [blob["business_id"]]
        highlights += [blob["highlights"]]
        delivery_or_takeout += [blob["delivery or takeout"]]
        Grubhub_enabled += [blob["Grubhub enabled"]]
        Call_To_Action_enabled += [blob["Call To Action enabled"]]
        Request_a_Quote_Enabled += [blob["Request a Quote Enabled"]]
        Covid_Banner += [blob["Covid Banner"]]
        Temporary_Closed_Until += [blob["Temporary Closed Until"]]
        Virtual_Services_Offered += [blob["Virtual Services Offered"]]

        
convid19_business = pd.DataFrame(
{"business_id": business_ids, "highlights": highlights, "delivery_or_takeout": delivery_or_takeout, "Grubhub_enabled": Grubhub_enabled, "Grubhub_enabled": Grubhub_enabled, \
"Call_To_Action_enabled": Call_To_Action_enabled, "Request_a_Quote_Enabled": Request_a_Quote_Enabled, "Covid_Banner": Covid_Banner, \
"Temporary_Closed_Until":Temporary_Closed_Until, "Virtual_Services_Offered":Virtual_Services_Offered}
)


In [ ]:
convid19_business.head()

**Collaborative filtering**

In [ ]:
import numpy as np
import pandas as pd
import json

from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate
from surprise.model_selection import KFold
from surprise import KNNBasic


class CFRecommender():

    def __init__(self):
        return None

    def read_data(self, df):
        df_stars = df[['user_id', 'business_id', 'stars']]
        # A reader is needed with rating_scale param
        reader = Reader(rating_scale=(1, 5))
        # The columns must correspond to user id, item id and ratings (in that order).
        data = Dataset.load_from_df(df_stars, reader)
        
        return data

    def CF_baseline(self, data):       

        # KFold
        kf = KFold(random_state=42)
        algo = KNNBasic(k=25,min_k=3,sim_options={'user_based': True}) 
        out = cross_validate(algo, data, ['rmse', 'mae'], kf)
        mean_rmse = np.mean(out['test_rmse'])
        mean_mae = np.mean(out['test_mae'])
        total_time = np.sum(out['fit_time'])
        print ("The results of KNN-based Recommender: ")
        print ("RMSE: %.3f"%mean_rmse)
        print ("MAE: %.3f"%mean_mae)
        print ("Total running time: %.2f s"%total_time)


if __name__ == "__main__":
    # read data
    df = pd.read_csv("/content/drive/My Drive/yelp-rs/dataset/last_1_years_restaurant_reviews.csv")
    
    modelbased = CFRecommender()
    data = modelbased.read_data(df)
    modelbased.CF_baseline(data)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
The results of KNN-based Recommender: 
RMSE: 1.495
MAE: 1.268
Total running time: 112.26 s


**SVD**

In [ ]:
import numpy as np
import pandas as pd
import json

# Model based recommender system using surprise
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate
from surprise import SVD
from surprise.model_selection import KFold

class ModelBasedRecommender():

    def __init__(self):

        return None

    def read_data(self, df):

        df_stars = df[['user_id', 'business_id', 'stars']]
        # A reader is needed with rating_scale param
        reader = Reader(rating_scale=(1, 5))
        # The columns must correspond to user id, item id and ratings (in that order).
        data = Dataset.load_from_df(df_stars, reader)

        return data,df_stars

    def fit_SVD(self, data):

        # KFold
        kf = KFold(random_state=42)
        # cross validation using SVD
        out = cross_validate(SVD(), data, ['rmse', 'mae'], kf,verbose=True)
        mean_rmse = np.mean(out['test_rmse'])
        mean_mae = np.mean(out['test_mae'])
        total_time = np.sum(out['fit_time'])
        print ("The results of SVD Recommender: ")
        print ("RMSE: %.3f"%mean_rmse)
        print ("MAE: %.3f"%mean_mae)
        print ("Total running time: %.2f s"%total_time)

        

if __name__ == "__main__":
    # read data
    df = pd.read_csv("/content/drive/My Drive/yelp-rs/dataset/last_1_years_restaurant_reviews.csv")

    #Model based recommender system
    modelbased = ModelBasedRecommender()
    data,df_stars = modelbased.read_data(df)
    modelbased.fit_SVD(data)



Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.3969  1.3768  1.4092  1.4021  1.3837  1.3937  0.0119  
MAE (testset)     1.1372  1.1268  1.1515  1.1410  1.1291  1.1371  0.0088  
Fit time          2.13    2.13    2.14    2.13    2.14    2.13    0.01    
Test time         0.05    0.05    0.05    0.13    0.05    0.07    0.03    
The results of SVD Recommender: 
RMSE: 1.394
MAE: 1.137
Total running time: 10.67 s


  Feature_engineer

In [ ]:
import numpy as np
import pandas as pd
# skip all warnings
import warnings
warnings.filterwarnings('ignore')

from sklearn.feature_extraction.text import CountVectorizer
# use svd to reduce dimension
from sklearn.decomposition import TruncatedSVD
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score, train_test_split


def feature_engineer(df):

    # group by business_id, then average numerical features
    df_average = df.groupby(['business_id']).mean()
    # group by business_id, extract categories data
    categories_series = df.groupby(['business_id']).categories.apply(np.unique)
    # convert categories data to string
    categories_series = categories_series.str.join('')

    # business_id, categories table
    vectorizer = CountVectorizer()
    categories_mat = vectorizer.fit_transform(categories_series).toarray()
    categories = vectorizer.get_feature_names()
    df_categories = pd.DataFrame(categories_mat,
                             columns=categories, 
                             index=categories_series.index)

    # initialize a SVD
    svd = TruncatedSVD(n_components=75, random_state=42)
    # fit SVD on categories_mat
    svd.fit(categories_mat)
    # transform original data
    categories_svd = svd.transform(categories_mat)
    # create a new dataframe
    df_categories_svd = pd.DataFrame(categories_svd,
                                 index=categories_series.index)

    # join df_average and df_categories_svd
    df_business = df_average.join(df_categories_svd)

    return df_average,df_categories_svd,df_business


if __name__ == "__main__":

    # read data
    df = pd.read_csv("/content/drive/My Drive/yelp-rs/dataset/last_1_years_restaurant_reviews.csv")
    # prepare for modeling
    df_average,df_categories_svd,data = feature_engineer(df)

    # get X and y
    target = 'stars'
    features = [x for x in data.columns if x not in ['avg_stars','stars']]
    y = data[target].values >= 4.0
    X = data[features].values

    # train test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    # candidate models
    models = [LogisticRegression(), 
              DecisionTreeClassifier(), 
              RandomForestClassifier(),
              AdaBoostClassifier(),
              XGBClassifier()]
    
    labels = ['LogisticRegression','DecisionTreeClassifier','RandomForestClassifier','AdaBoostClassifier','XGBClassifier']
    
    print('5-fold cross validation:\n')
    # train models
    for model, label in zip(models,labels):
        scores = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy')
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        #print(y_pred)
        print("Accuracy: %0.4f (+/- %0.4f) [%s]" % (scores.mean(), scores.std(), label))


5-fold cross validation:

Accuracy: 0.6194 (+/- 0.0228) [LogisticRegression]
Accuracy: 0.5602 (+/- 0.0052) [DecisionTreeClassifier]
Accuracy: 0.5981 (+/- 0.0087) [RandomForestClassifier]
Accuracy: 0.6078 (+/- 0.0077) [AdaBoostClassifier]
Accuracy: 0.6074 (+/- 0.0189) [XGBClassifier]


In [ ]:
data.head()

,avg_stars,stars,useful,funny,cool,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74
business_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
--9e1ONYQuAa-CB_Rrw7Tw,4.0,3.363636,0.636364,0.090909,0.090909,0.719213,0.043589,-0.125214,-0.693685,-0.229573,-0.065165,-0.033492,0.235846,0.024949,-0.212596,0.110927,-0.124858,-0.150895,0.013397,-0.002851,-0.207688,-0.828878,0.561935,-0.633969,0.172821,-0.265573,0.396962,0.070289,0.061575,0.150410,-0.107400,-0.101525,-0.012036,0.107380,0.314326,-0.344883,0.101335,-0.080615,0.189405,0.190779,-0.124589,0.015992,-0.013707,-0.036788,-0.051166,0.063681,0.021543,-0.027831,0.001853,0.065507,0.039837,-0.101650,-0.025073,-0.046521,0.039164,-0.123385,0.007806,0.123650,-0.288805,0.147168,-0.069503,-0.302826,-0.015731,0.006950,-0.133864,0.760812,0.243874,-0.186351,-0.123368,-0.144041,-0.397034,-0.496375,0.121945,-0.228947,-0.172414,-0.198578,0.075576,-0.120278,0.018199,-0.169232
-0RkJ_uIduNLWQrphbADRw,4.0,3.857143,0.642857,0.428571,0.285714,1.485237,-0.673197,-0.373417,0.164949,-0.226635,1.613198,-0.367631,-0.164478,0.517850,0.071500,0.620624,0.054916,0.172598,-0.246500,-0.173240,-0.255423,-0.019151,-0.122649,-0.009398,0.112518,-0.144390,-0.028347,0.138232,-0.040648,0.009870,-0.045602,0.044674,-0.089487,0.338878,-0.220865,-0.105082,-0.095859,-0.352965,0.041913,-0.046016,-0.025063,-0.117544,-0.036288,-0.095319,0.015172,-0.050009,0.004926,0.028666,-0.081845,0.049546,-0.002565,-0.011223,-0.026597,0.019467,-0.039147,-0.036709,0.022731,0.055172,-0.045421,0.020290,0.038334,-0.001678,0.029526,-0.008354,-0.026831,-0.044007,-0.030255,0.076503,-0.023623,-0.010462,-0.037229,0.023545,0.078076,-0.018729,0.017420,-0.095891,-0.097898,-0.018363,0.001246,0.022175
-1m9o3vGRA8IBPNvNqKLmA,4.5,4.090909,1.977273,1.045455,1.681818,1.896610,1.980260,0.458432,0.234280,-0.131408,-0.274242,0.133601,0.231758,-0.048730,-0.331230,0.104328,0.154323,0.001005,-0.589729,0.466921,0.035791,-0.776738,0.274852,-0.792506,0.030546,-0.357760,0.263351,0.126210,0.012284,-0.228120,-0.028419,-0.040465,-0.071537,0.207403,0.322410,-0.422127,0.331004,-0.085936,-0.152726,-0.081760,0.058614,-0.071507,0.018485,0.220251,-0.012050,-0.039069,0.196394,-0.067066,-0.111944,-0.045291,0.212777,0.034444,-0.083327,0.191773,0.189874,-0.004259,-0.080109,-0.016465,0.043904,-0.034814,0.050490,0.019155,-0.058902,0.029694,0.032567,-0.039664,-0.102712,0.003158,-0.011429,-0.021083,0.041335,-0.038394,0.007992,0.029947,0.092237,-0.004281,0.017460,0.069411,0.015323,0.043455
-2nfJ8yK54A7Md2RzmMR4g,5.0,5.000000,0.000000,0.000000,0.000000,2.026325,-0.554839,1.029416,0.340695,-0.181489,0.078207,-0.184030,0.579496,-0.418481,0.308033,-0.038909,0.007841,-0.120938,0.532875,0.572571,-0.768864,-0.111772,-0.341731,0.199325,-0.157212,0.066280,-0.172451,-0.049048,-0.320641,0.052225,-0.112909,-0.013443,-0.104358,-0.054999,-0.061473,-0.010495,0.086019,-0.072085,-0.175502,0.027677,-0.029489,-0.200864,-0.000647,-0.162224,-0.014533,0.306741,0.032140,0.045601,-0.095218,0.045398,0.012205,-0.387013,0.049265,0.075067,-0.049818,0.108506,0.129804,-0.027201,-0.174475,-0.025359,0.062937,-0.062287,-0.043347,0.127505,-0.029771,-0.270158,-0.020708,0.115427,0.083321,-0.060283,0.041553,-0.228189,0.076368,-0.214360,-0.098034,0.178161,0.114605,0.007445,0.037523,-0.079140
-3cJ2k_iwauMpy1WfI4NrA,4.5,4.600000,0.133333,0.066667,0.133333,0.663650,-0.026422,-0.115459,-0.669812,-0.267652,0.007296,-0.118433,0.174397,0.080546,-0.120978,-0.043614,-0.032946,-0.084893,-0.051189,-0.106454,0.125041,-0.197141,-0.082782,0.205308,-0.090652,0.051699,-0.150845,0.077879,0.088042,-0.057733,-0.063479,-0.159642,-0.041597,-0.087230,0.226467,0.124670,-0.243319,-0.227044,0.360826,-0.423628,-0.163176,-0.242278,0.743718,0.692476,0.369279,-0.030614,0.115580,-0.314657,0.051195,0.432821,-0.062242,-0.317475,0.193153,-0.480771,0.147439,

Hybrid Rcommender

In [9]:
import numpy as np
import pandas as pd
import json

from surprise import Dataset
from surprise import Reader
from surprise import SVD
from surprise import KNNBasic


class Hybrid_Recommender():

    def __init__(self):
        return None

    def read_data(self, df):
        df_stars = df[['user_id', 'business_id', 'stars']]
        # A reader is needed with rating_scale param
        reader = Reader(rating_scale=(1, 5))
        # The columns must correspond to user id, item id and ratings (in that order).
        data = Dataset.load_from_df(df_stars, reader)

        return data
        
    def train(self, data):       

        trainset = data.build_full_trainset()
        algo1 = KNNBasic(k=25,min_k=3,sim_options={'user_based': True})
        algo2 = SVD()
        algo1.fit(trainset)
        algo2.fit(trainset)
        return algo1,algo2

    def predict(self, algo1, algo2, uid, iid):       
        pred1 = algo1.predict(uid, iid, verbose=True)
        pred2 = algo2.predict(uid, iid, verbose=True)
        return self.weighted_hybrid(pred1,pred2)

    def weighted_hybrid(self,pred1,pred2):
        p1 = list(str(pred1).split(' '))
        for i in range(len(p1)):
          if p1[i]=='est':
            p1_ = float(p1[i+2])
            break
        
        p2 = list(str(pred2).split(' '))
        for i in range(len(p2)):
          if p2[i]=='est':
            p2_ = float(p2[i+2])
            break
        
        w1 = 0.5
        w2 = 0.5
        return w1*p1_ + w2*p2_


Interface 

In [10]:
if __name__ == "__main__":
    # read data
    df = pd.read_csv("/content/drive/My Drive/yelp-rs/dataset/last_1_years_restaurant_reviews.csv")
    
    Hybridmodel = Hybrid_Recommender()
    data = Hybridmodel.read_data(df)
    algo1,algo2 = Hybridmodel.train(data)
    print("interface")
    #uid = 'wbbG7CXQ70pcHv0GPFNg0g'
    #iid = '9e1ONYQuAa-CB_Rrw7Tw'
    uid = input("uid:")
    iid = input("iid:")
    print("Hybrid Rcommender result: ",Hybridmodel.predict(algo1,algo2, uid, iid))

Computing the msd similarity matrix...
Done computing similarity matrix.
interface
uid:wbbG7CXQ70pcHv0GPFNg0g
iid:9e1ONYQuAa-CB_Rrw7Tw
user: wbbG7CXQ70pcHv0GPFNg0g item: 9e1ONYQuAa-CB_Rrw7Tw r_ui = None   est = 3.84   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}
user: wbbG7CXQ70pcHv0GPFNg0g item: 9e1ONYQuAa-CB_Rrw7Tw r_ui = None   est = 3.96   {'was_impossible': False}
Hybrid Rcommender result:  3.9
